In [1]:
import pandas as pd
import pickle
import argparse
import os
import glob
import pydicom
import sys

/tmp/ipykernel_2849961/1002562574.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import nibabel as nib
import numpy as np
import pandas as pd
import os
import glob

In [3]:
combined_roi_dict = {
    'Left-Cingulate': {
        'ctx-lh-caudalanteriorcingulate',
        'ctx-lh-rostralanteriorcingulate',
        'ctx-lh-isthmuscingulate',
        'ctx-lh-posteriorcingulate',
    },
    'Right-Cingulate': {
        'ctx-rh-caudalanteriorcingulate',
        'ctx-rh-rostralanteriorcingulate',
        'ctx-rh-isthmuscingulate',
        'ctx-rh-posteriorcingulate',
    },
    'Left-Frontal': {
        'ctx-lh-caudalmiddlefrontal',
        'ctx-lh-lateralorbitofrontal',
        'ctx-lh-medialorbitofrontal',
        'ctx-lh-parsopercularis',
        'ctx-lh-parsorbitalis',
        'ctx-lh-parstriangularis',
        'ctx-lh-precentral',
        'ctx-lh-rostralmiddlefrontal',
        'ctx-lh-superiorfrontal',
        'ctx-lh-paracentral'
    },
    'Right-Frontal': {
        'ctx-rh-caudalmiddlefrontal',
        'ctx-rh-lateralorbitofrontal',
        'ctx-rh-medialorbitofrontal',
        'ctx-rh-parsopercularis',
        'ctx-rh-parsorbitalis',
        'ctx-rh-parstriangularis',
        'ctx-rh-precentral',
        'ctx-rh-rostralmiddlefrontal',
        'ctx-rh-superiorfrontal',
        'ctx-rh-paracentral'
    },
    'Left-Temporal': {
        'ctx-lh-entorhinal',
        'ctx-lh-fusiform',
        'ctx-lh-inferiortemporal',
        'ctx-lh-middletemporal',
        'ctx-lh-superiortemporal',
        'ctx-lh-transversetemporal',
        'ctx-lh-parahippocampal',
        'Left-Hippocampus',
        'Left-Amygdala',
    },
    'Right-Temporal': {
        'ctx-rh-entorhinal',
        'ctx-rh-fusiform',
        'ctx-rh-inferiortemporal',
        'ctx-rh-middletemporal',
        'ctx-rh-superiortemporal',
        'ctx-rh-transversetemporal',
        'ctx-rh-parahippocampal',
        'Right-Hippocampus',
        'Right-Amygdala',
    },
    'Left-Parietal': {
        'ctx-lh-inferiorparietal',
        'ctx-lh-postcentral',
        'ctx-lh-superiorparietal',
        'ctx-lh-supramarginal',
        'ctx-lh-precuneus'
    },
    'Right-Parietal': {
        'ctx-rh-inferiorparietal',
        'ctx-rh-postcentral',
        'ctx-rh-superiorparietal',
        'ctx-rh-supramarginal',
        'ctx-rh-precuneus'
    },
    'Left-Occipital': {
        'ctx-lh-cuneus',
        'ctx-lh-lateraloccipital',
        'ctx-lh-lingual',
        'ctx-lh-pericalcarine'
    },
    'Right-Occipital': {
        'ctx-rh-cuneus',
        'ctx-rh-lateraloccipital',
        'ctx-rh-lingual',
        'ctx-rh-pericalcarine'
    },
    'Left-Insula':{
        'ctx-lh-insula'
    },
    'Right-Insula': {
        'ctx-rh-insula'
    },
    'Left-Cerebellum':{
        'Left-Cerebellum-White-Matter',
        'Left-Cerebellum-Cortex'
    },
    'Right-Cerebellum': {
        'Right-Cerebellum-White-Matter',
        'Right-Cerebellum-Cortex'
    },
    'Left-Combined-Cerebral-WM': {
        'Left-Cerebral-White-Matter',
        'Left-WM-hypointensities'
    },
    'Right-Combined-Cerebral-WM': {
        'Right-Cerebral-White-Matter',
        'Right-WM-hypointensities'
    },
    'Left-Total-Lateral-Ventricle': {
        'Left-Lateral-Ventricle',
        'Left-Inf-Lat-Vent'
    },
    'Right-Total-Lateral-Ventricle': {
        'Right-Lateral-Ventricle',
        'Right-Inf-Lat-Vent'
    },
}
gray_matter_index = {
    'left_cortical_gm': {
        'left_cingulate',
        'left_frontal',
        'left_temporal',
        'left_parietal',
        'left_occipital',
        'left_insula'
    },
    'right_cortical_gm': {
        'right_cingulate',
        'right_frontal',
        'right_temporal',
        'right_parietal',
        'right_occipital',
        'right_insula'
    }
}

combined_roi_display_dict = {
    'Left-Cingulate': {
        'ctx-lh-caudalanteriorcingulate',
        'ctx-lh-rostralanteriorcingulate',
        'ctx-lh-isthmuscingulate',
        'ctx-lh-posteriorcingulate',
    },
    'Right-Cingulate': {
        'ctx-rh-caudalanteriorcingulate',
        'ctx-rh-rostralanteriorcingulate',
        'ctx-rh-isthmuscingulate',
        'ctx-rh-posteriorcingulate',
    },
    'Left-Frontal': {
        'ctx-lh-caudalmiddlefrontal',
        'ctx-lh-lateralorbitofrontal',
        'ctx-lh-medialorbitofrontal',
        'ctx-lh-parsopercularis',
        'ctx-lh-parsorbitalis',
        'ctx-lh-parstriangularis',
        'ctx-lh-precentral',
        'ctx-lh-rostralmiddlefrontal',
        'ctx-lh-superiorfrontal',
        'ctx-lh-paracentral'
    },
    'Right-Frontal': {
        'ctx-rh-caudalmiddlefrontal',
        'ctx-rh-lateralorbitofrontal',
        'ctx-rh-medialorbitofrontal',
        'ctx-rh-parsopercularis',
        'ctx-rh-parsorbitalis',
        'ctx-rh-parstriangularis',
        'ctx-rh-precentral',
        'ctx-rh-rostralmiddlefrontal',
        'ctx-rh-superiorfrontal',
        'ctx-rh-paracentral'
    },
    'Left-Temporal': {
        'ctx-lh-entorhinal',
        'ctx-lh-fusiform',
        'ctx-lh-inferiortemporal',
        'ctx-lh-middletemporal',
        'ctx-lh-superiortemporal',
        'ctx-lh-transversetemporal',
        'ctx-lh-parahippocampal',
    },
    'Right-Temporal': {
        'ctx-rh-entorhinal',
        'ctx-rh-fusiform',
        'ctx-rh-inferiortemporal',
        'ctx-rh-middletemporal',
        'ctx-rh-superiortemporal',
        'ctx-rh-transversetemporal',
        'ctx-rh-parahippocampal',
    },
    'Left-Parietal': {
        'ctx-lh-inferiorparietal',
        'ctx-lh-postcentral',
        'ctx-lh-superiorparietal',
        'ctx-lh-supramarginal',
        'ctx-lh-precuneus'
    },
    'Right-Parietal': {
        'ctx-rh-inferiorparietal',
        'ctx-rh-postcentral',
        'ctx-rh-superiorparietal',
        'ctx-rh-supramarginal',
        'ctx-rh-precuneus'
    },
    'Left-Occipital': {
        'ctx-lh-cuneus',
        'ctx-lh-lateraloccipital',
        'ctx-lh-lingual',
        'ctx-lh-pericalcarine'
    },
    'Right-Occipital': {
        'ctx-rh-cuneus',
        'ctx-rh-lateraloccipital',
        'ctx-rh-lingual',
        'ctx-rh-pericalcarine'
    },
    'Left-Insula':{
        'ctx-lh-insula'
    },
    'Right-Insula': {
        'ctx-rh-insula'
    },
    'Left-Cerebellum':{
        'Left-Cerebellum-White-Matter',
        'Left-Cerebellum-Cortex'
    },
    'Right-Cerebellum': {
        'Right-Cerebellum-White-Matter',
        'Right-Cerebellum-Cortex'
    },
    'Left-Combined-Cerebral-WM': {
        'Left-Cerebral-White-Matter',
        'Left-WM-hypointensities'
    },
    'Right-Combined-Cerebral-WM': {
        'Right-Cerebral-White-Matter',
        'Right-WM-hypointensities'
    }
}

In [4]:
# 경로만 지정!
#base_dir = '/mnt/drwelly1/Team_Planning/Brain_Volumetry/OASIS1_after_swift_20240403'
base_dir = '/home/limseoyoung/drwelly1/Team_Planning/Brain_Volumetry/OASIS2_after_swift_20240403'


def calculate_volume(nii_path):
    img = nib.load(nii_path) 
    data = img.get_fdata()
    header = img.header
    voxel_volume = np.prod(header.get_zooms())
    unique_elements, counts_elements = np.unique(data, return_counts=True)
    volumes = counts_elements * voxel_volume
    return dict(zip(unique_elements.astype(int), volumes))


In [5]:
import os
import pandas as pd



volume_data = []

for subject_dir in os.listdir(base_dir):
    full_subject_dir = os.path.join(base_dir, subject_dir)
    if not os.path.isdir(full_subject_dir) or 'FS222' not in subject_dir:
        continue
    
    double_subject_dir = os.path.join(full_subject_dir, subject_dir)
    if not os.path.isdir(double_subject_dir):
        continue
    
    mri_dir = os.path.join(double_subject_dir, 'mri')
    nii_path = os.path.join(mri_dir, 'aparc.DKTatlas+aseg.deep.nii.gz')
    
    if os.path.isfile(nii_path):
        try:
            vol_data = calculate_volume(nii_path)
            vol_data["subject"] = subject_dir
            volume_data.append(vol_data)
        except OSError as e:
            print(f"Error processing file {nii_path}, skipped  Error: {e}")
    else:
        print(f"File not found, skipped: {nii_path}")

volume_df = pd.DataFrame(volume_data)

In [6]:
volume_df 

,0,2,4,5,7,8,10,11,12,13,...,2025,2026,2027,2028,2029,2030,2031,2034,2035,subject
0,1.575305e+07,195613.583009,19908.405696,852.011987,10247.124843,46590.718288,7106.616891,5072.626922,4918.276925,1887.185971,...,9296.328858,1843.281972,9478.118855,21946.854664,9813.229850,15243.948767,8736.895866,889.055986,5707.176913,OAS2_0076_MR3_mpr-2_Tag_pr100_sr100_ppf100_spf...
1,1.581092e+07,210513.159781,11774.846820,534.736992,10673.473837,42878.772344,6156.163906,3173.778951,4081.356938,1756.159973,...,7654.387883,1701.965974,7026.011893,22507.316656,9061.716861,13349.559796,8761.591866,922.669986,5640.291914,OAS2_0060_MR2_mpr-2_Tag_pr100_sr100_ppf100_spf...
2,1.554688e+07,252897.326133,33311.130491,938.790986,12272.196812,50431.289229,7633.807883,4414.066932,5260.933920,2133.459967,...,11972.071817,2625.321960,12403.908810,27088.767586,10998.637832,13289.534797,10593.897838,798.160988,7083.635892,OAS2_0161_MR1_mpr-4_Tag_pr100_sr100_ppf100_spf...
3,1.576462e+07,217201.659679,10012.169847,645.868990,12802.817804,46812.296284,7156.351891,3534.614946,5006.427923,2115.280968,...,8931.033863,2006.206969,9809.456850,21299.613674,9590.279853,12172.383814,8714.600867,794.044988,5051.017923,OAS2_0079_MR1_mpr-4_Tag_pr100_sr100_ppf100_spf...
4,1.596496e+07,145735.552771,13725.830790,543.311992,10584.979838,39222.049400,6137.984906,3815.188942,4738.544928,1739.695973,...,6842.506895,2059.714969,8726.605867,20994.343679,7614.599884,11005.497832,6615.440899,694.917989,5221.488920,OAS2_0004_MR2_mpr-3_Tag_pr100_sr100_ppf100_spf...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1361,1.570728e+07,214700.503717,14475.971779,985.438985,14625.519776,61350.008062,7703.093882,3008.452954,4149.613937,2073.777968,...,9180.051860,2122.140968,7873.564880,24240.838629,8848.027865,14619.345776,10435.431840,886.997986,5823.453911,OAS2_0028_MR1_mpr-1_Tag_pr100_sr100_ppf100_spf...
1362,1.567379e+07,235698.963396,10952.675833,540.910992,12276.998812,53286.421185,7169.042890,3614.876945,5504.463916,2198.629966,...,9202.346859,2238.417966,9408.832856,25027.337617,8407.615871,14442.014779,7313.445888,723.386989,5219.087920,OAS2_0137_MR1_mpr-3_Tag_pr100_sr100_ppf100_spf...
1363,1.586254e+07,180931.125233,17503.632732,1295.167980,10089.687846,41623.735363,6053.949907,2568.726961,3634.427944,1371.313979,...,8371.257872,1900.219971,7737.050882,23817.919636,8177.119875,12061.251816,8673.440867,763.517988,5497.946916,OAS2_0026_MR1_mpr-4_Tag_pr100_sr100_ppf100_spf...
1364,1.569988e+07,227852.152516,21212.491676,839.320987,11637.303822,45458.132305,7026.697893,3589.151945,4002.123939,1704.709974,...,8006.305878,2329.312964,8843.911865,27310.688582,9175.935860,13615.727792,7926.386879,787.870988,6145.530906,OAS2_0100_MR2_mpr-2_Tag_pr100_sr100_ppf100_spf...


In [7]:
volume_df['subject'] = volume_df['subject'].str.replace('_Tag_pr100_sr100_ppf100_spf100_ipf100_sipf179_FS222', '')


In [8]:

volume_df_mpr1 = volume_df[volume_df['subject'].str.contains('MR1')]
volume_df_mpr2 = volume_df[volume_df['subject'].str.contains('MR2')]
volume_df_mpr3 = volume_df[volume_df['subject'].str.contains('MR3')]
volume_df_mpr4 = volume_df[volume_df['subject'].str.contains('MR4')]

In [9]:
volume_df_mpr4.shape

(47, 97)

In [10]:
volume_df_mpr1 

,0,2,4,5,7,8,10,11,12,13,...,2025,2026,2027,2028,2029,2030,2031,2034,2035,subject
2,1.554688e+07,252897.326133,33311.130491,938.790986,12272.196812,50431.289229,7633.807883,4414.066932,5260.933920,2133.459967,...,11972.071817,2625.321960,12403.908810,27088.767586,10998.637832,13289.534797,10593.897838,798.160988,7083.635892,OAS2_0161_MR1_mpr-4
3,1.576462e+07,217201.659679,10012.169847,645.868990,12802.817804,46812.296284,7156.351891,3534.614946,5006.427923,2115.280968,...,8931.033863,2006.206969,9809.456850,21299.613674,9590.279853,12172.383814,8714.600867,794.044988,5051.017923,OAS2_0079_MR1_mpr-4
5,1.598560e+07,155761.442618,9047.653862,487.745993,7481.858886,36065.077448,6208.299905,3522.266946,4316.997934,1696.820974,...,6561.246900,2040.506969,7420.461887,20156.737692,7183.105890,12335.994811,6556.787900,811.880988,5953.107909,OAS2_0158_MR1_mpr-2
7,1.577893e+07,208722.699808,32519.829503,3707.829943,10116.784845,37997.882419,6526.260900,3460.869947,4269.663935,1832.305972,...,7395.079887,2167.416967,9546.718854,21247.820675,9268.888858,11014.415832,8123.611876,902.432986,4923.764925,OAS2_0014_MR1_mpr-4
9,1.580029e+07,195797.774006,14859.102773,622.201990,10052.300846,48347.221261,6957.754894,3657.751944,4339.978934,1844.996972,...,9295.299858,2018.554969,9584.448853,24081.000632,10253.641843,13555.359793,8392.180872,997.100985,4825.666926,OAS2_0116_MR1_mpr-3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1356,1.597241e+07,143328.035808,13175.658799,575.210991,10273.192843,38767.917407,5940.073909,3837.826941,4840.072926,1776.739973,...,6913.507894,1928.688971,8336.957873,20914.081680,8022.083877,10591.839838,6599.662899,682.226990,5442.723917,OAS2_0004_MR1_mpr-4
1360,1.584097e+07,165707.756466,39716.655393,1241.316981,11228.790828,53204.101186,6293.020904,3413.192948,3488.309947,1789.430973,...,9073.378861,2082.352968,8124.983876,18780.621713,8756.789866,13676.781791,7362.837887,735.734989,5249.614920,OAS2_0034_MR1_mpr-2
1361,1.570728e+07,214700.503717,14475.971779,985.438985,14625.519776,61350.008062,7703.093882,3008.452954,4149.613937,2073.777968,...,9180.051860,2122.140968,7873.564880,24240.838629,8848.027865,14619.345776,10435.431840,886.997986,5823.453911,OAS2_0028_MR1_mpr-1
1362,1.567379e+07,235698.963396,10952.675833,540.910992,12276.998812,53286.421185,7169.042890,3614.876945,5504.463916,2198.629966,...,9202.346859,2238.417966,9408.832856,25027.337617,8407.615871,14442.014779,7313.445888,723.386989,5219.087920,OAS2_0137_MR1_mpr-3


In [11]:

def extract_number(subj):
    parts = subj.split('_')  # '_'를 기준으로 문자열 분리
    if len(parts) > 1:
        number_part = parts[1]  # "OAS1_" 다음 오는 부분을 선택
        # 숫자만 추출
        numbers = ''.join(filter(str.isdigit, number_part))
        return int(numbers) if numbers.isdigit() else None
    return None

In [12]:
lut_path = '/mnt/drwelly1/Team_Planning/Brain_Volumetry/Algorithms/freesurfer/FreeSurferColorLUT.txt'
# labeling 불러오면됨

In [13]:
def load_freesurfer_lut(lut_path):
    lut = {}
    with open(lut_path, 'r') as file:
        for line in file:
            if line.strip() and not line.startswith('#'):
                parts = line.split()
                if parts[0].isdigit():
                    lut[int(parts[0])] = ' '.join(parts[1:])
    return lut

freesurfer_lut = load_freesurfer_lut(lut_path)

In [14]:
# 모든 컬럼 이름을 변경
new_columns = [freesurfer_lut.get(int(col), 'Unknown') if col != 'subject' else 'subject' for col in volume_df_mpr1 .columns]


volume_df_mpr1.columns = new_columns

In [15]:

def extract_number(subj):
    parts = subj.split('_')  # '_'를 기준
    if len(parts) > 1:
        number_part = parts[1]  
        # 숫자만 
        numbers = ''.join(filter(str.isdigit, number_part))
        return int(numbers) if numbers.isdigit() else None
    return None

In [16]:

volume_df_mpr1 ['sort_key'] = volume_df_mpr1['subject'].apply(extract_number) 
volume_df_sorted1 = volume_df_mpr1 .sort_values(by='sort_key').drop(columns=['sort_key']) 


/tmp/ipykernel_2849961/1083419085.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  volume_df_mpr1 ['sort_key'] = volume_df_mpr1['subject'].apply(extract_number)


In [17]:
# 모든 컬럼 이름을 변경
new_columns = [freesurfer_lut.get(int(col), 'Unknown') if col != 'subject' else 'subject' for col in volume_df_mpr2 .columns]


volume_df_mpr2.columns = new_columns

In [18]:
# 모든 컬럼 이름을 변경
new_columns = [freesurfer_lut.get(int(col), 'Unknown') if col != 'subject' else 'subject' for col in volume_df_mpr3.columns]


volume_df_mpr3.columns = new_columns

In [19]:

volume_df_mpr3['sort_key'] = volume_df_mpr3['subject'].apply(extract_number)  # 정렬 키 생성
volume_df_sorted3 = volume_df_mpr3.sort_values(by='sort_key').drop(columns=['sort_key'])  # 정렬 및 정렬 키 컬럼 삭제


/tmp/ipykernel_2849961/593579216.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  volume_df_mpr3['sort_key'] = volume_df_mpr3['subject'].apply(extract_number)  # 정렬 키 생성


In [20]:
# 모든 컬럼 이름을 변경
new_columns = [freesurfer_lut.get(int(col), 'Unknown') if col != 'subject' else 'subject' for col in volume_df_mpr4.columns]


volume_df_mpr4.columns = new_columns

In [21]:
volume_df_mpr4['sort_key'] = volume_df_mpr4['subject'].apply(extract_number) 
volume_df_sorted4 = volume_df_mpr4.sort_values(by='sort_key').drop(columns=['sort_key'])  


/tmp/ipykernel_2849961/2856656302.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  volume_df_mpr4['sort_key'] = volume_df_mpr4['subject'].apply(extract_number)


In [23]:

#volume_df['sort_key'] = volume_df['subject'].apply(extract_number)  # 정렬 키 생성
#df_final= volume_df.sort_values(by='sort_key').drop(columns=['sort_key'])  # 정렬 및 정렬 키 컬럼 삭제


In [24]:
#df_final

In [25]:
# 모든 컬럼 이름을 변경
#new_columns = [freesurfer_lut.get(int(col), 'Unknown') if col != 'subject' else 'subject' for col in df_final .columns]


#df_final.columns = new_columns

In [26]:
#df_final

In [27]:
def update_df_columns_keep_others(df, roi_dict):
    # 합쳐진 컬럼들의 원본 컬럼 이름을 저장할 리스트
    columns_to_remove = []
    
    # ROI 딕셔너리에 기반하여 합치는 작업을 수행
    for roi, sub_columns in roi_dict.items():
        # 해당 ROI에 속하는 데이터프레임의 컬럼들 찾기
        relevant_columns = [col for col in df.columns if col.split(' ')[0] in sub_columns]
        
        # 해당 컬럼들이 데이터프레임에 있는 경우, 합쳐진 컬럼 생성
        if relevant_columns:
            # 합산 전에 NaN 값을 0으로 변환
            df[roi] = df[relevant_columns].fillna(0).sum(axis=1)
            # 원본 컬럼 이름을 제거 리스트에 추가
            columns_to_remove.extend(relevant_columns)
    
    # 모든 ROI 컬럼들을 포함하는 리스트 생성
    all_roi_columns = set()
    for sub_columns in roi_dict.values():
        all_roi_columns.update(sub_columns)
    
    # 제거할 컬럼 리스트에서, 딕셔너리에 정의된 컬럼만 실제로 제거
    columns_to_remove = set([col for col in columns_to_remove if col.split(' ')[0] in all_roi_columns])
    
    # 실제로 제거할 컬럼들을 제거
    df.drop(columns=columns_to_remove, inplace=True, errors='ignore')
    
    return df

# 함수 실행하여 데이터프레임 업데이트
new_df1 = update_df_columns_keep_others(volume_df_mpr1.copy(), combined_roi_dict)
new_df2 = update_df_columns_keep_others(volume_df_mpr2.copy(), combined_roi_dict)
new_df3 = update_df_columns_keep_others(volume_df_mpr3.copy(), combined_roi_dict)
new_df4 = update_df_columns_keep_others(volume_df_mpr4.copy(), combined_roi_dict)


In [28]:
#new_final_df =  update_df_columns_keep_others(df_final.copy(), combined_roi_dict)
#new_final_df

In [29]:
#new_final_df.columns

In [30]:

brain_areas = set()
left_right_columns = []

for col in new_df1.columns:
    if col.startswith('Left-') or col.startswith('Right-'):
        left_right_columns.append(col)
        brain_area = col.split('-', 1)[1]
        brain_areas.add(brain_area)


for area in brain_areas:
    left_col = f'Left-{area}'
    right_col = f'Right-{area}'
    new_df1[area] = new_df1.get(left_col, 0) + new_df1.get(right_col, 0)

volume_df_sorted1_final = new_df1.drop(columns=[col for col in left_right_columns if col not in ['subject']])


In [31]:
volume_df_sorted1_final= volume_df_sorted1_final.drop(columns = ['sort_key','Unknown 0 0 0 0','choroid-plexus 0 200 200 0','choroid-plexus 0 200 221 0','WM-hypointensities 200 70 255 0']) 

In [32]:
volume_df_sorted1_final.shape

(582, 21)

In [33]:
brain_areas = set()
left_right_columns = []

for col in new_df2.columns:
    if col.startswith('Left-') or col.startswith('Right-'):
        left_right_columns.append(col)
        brain_area = col.split('-', 1)[1]
        brain_areas.add(brain_area)


for area in brain_areas:
    left_col = f'Left-{area}'
    right_col = f'Right-{area}'
    new_df2[area] = new_df2.get(left_col, 0) + new_df2.get(right_col, 0)

volume_df_sorted2_final = new_df2.drop(columns=[col for col in left_right_columns if col not in ['subject']])


In [34]:
volume_df_sorted2_final= volume_df_sorted2_final.drop(columns = ['Unknown 0 0 0 0','choroid-plexus 0 200 200 0','choroid-plexus 0 200 221 0','WM-hypointensities 200 70 255 0']) 

In [35]:
#volume_df_sorted2_final

In [36]:
brain_areas = set()
left_right_columns = []

for col in new_df3.columns:
    if col.startswith('Left-') or col.startswith('Right-'):
        left_right_columns.append(col)
        brain_area = col.split('-', 1)[1]
        brain_areas.add(brain_area)


for area in brain_areas:
    left_col = f'Left-{area}'
    right_col = f'Right-{area}'
    new_df3[area] = new_df3.get(left_col, 0) + new_df3.get(right_col, 0)

volume_df_sorted3_final = new_df3.drop(columns=[col for col in left_right_columns if col not in ['subject']])


In [37]:
volume_df_sorted3_final= volume_df_sorted3_final.drop(columns = ['sort_key','Unknown 0 0 0 0','choroid-plexus 0 200 200 0','choroid-plexus 0 200 221 0','WM-hypointensities 200 70 255 0']) 

In [38]:

brain_areas = set()
left_right_columns = []

for col in new_df4.columns:
    if col.startswith('Left-') or col.startswith('Right-'):
        left_right_columns.append(col)
        brain_area = col.split('-', 1)[1]
        brain_areas.add(brain_area)


for area in brain_areas:
    left_col = f'Left-{area}'
    right_col = f'Right-{area}'
    new_df4[area] = new_df4.get(left_col, 0) + new_df4.get(right_col, 0)

volume_df_sorted4_final = new_df4.drop(columns=[col for col in left_right_columns if col not in ['subject']])


In [39]:
volume_df_sorted4_final= volume_df_sorted4_final.drop(columns = ['sort_key','Unknown 0 0 0 0','choroid-plexus 0 200 200 0','choroid-plexus 0 200 221 0','WM-hypointensities 200 70 255 0']) 

In [40]:
volume_df_sorted1_final.to_csv('session1_oasis2_atropy_20240418_after_fastsurfer.csv')

In [41]:
volume_df_sorted2_final.to_csv('session2_oasis2_atropy_20240418_after_fastsurfer.csv')

In [42]:
volume_df_sorted3_final.to_csv('session3_oasis2_atropy_20240418_after_fastsurfer.csv')

In [43]:
volume_df_sorted4_final.to_csv('session4_oasis2_atropy_20240418_after_fastsurfer.csv')

In [44]:
pwd

'/home/limseoyoung/debug/For_tabulardata_testing'

In [58]:

brain_areas = set()
left_right_columns = []

for col in new_final_df.columns:
    if col.startswith('Left-') or col.startswith('Right-'):
        left_right_columns.append(col)
        brain_area = col.split('-', 1)[1]
        brain_areas.add(brain_area)


for area in brain_areas:
    left_col = f'Left-{area}'
    right_col = f'Right-{area}'
    new_final_df[area] = new_final_df.get(left_col, 0) + new_final_df.get(right_col, 0)

new_final_df_all = new_final_df.drop(columns=[col for col in left_right_columns if col not in ['subject']])


In [59]:

new_final_df_all 

,Unknown 0 0 0 0,3rd-Ventricle 204 182 142 0,4th-Ventricle 42 204 164 0,Brain-Stem 119 159 176 0,CSF 60 60 60 0,WM-hypointensities 200 70 255 0,subject,Pallidum 12 48 255 0,Parietal,Combined-Cerebral-WM,...,Pallidum 13 48 255 0,Accumbens-area 255 165 0 0,Caudate 122 186 220 0,Frontal,Thalamus 0 118 14 0,Cerebellum,choroid-plexus 0 200 200 0,Cingulate,Insula,choroid-plexus 0 200 221 0
1188,1.548241e+07,3233.803951,2464.454962,24578.007624,2618.461960,17578.063731,OAS2_0001_MR2_mpr-2,2179.421967,119006.935180,493245.654457,...,2446.961963,1263.611981,8901.535864,174925.881325,15759.134759,139501.527867,1020.081984,25433.449611,14458.478779,1085.937983
145,1.546988e+07,3206.363951,2414.033963,24679.192623,2534.426961,14991.843771,OAS2_0001_MR1_mpr-2,2251.794966,122255.488130,496441.042408,...,2471.314962,1340.443980,8865.863864,180615.908238,15780.400759,139542.001866,978.921985,26315.988598,14531.194778,1191.581982
82,1.548462e+07,3227.972951,2496.696962,24671.646623,2618.804960,17787.979728,OAS2_0001_MR2_mpr-3,2176.677967,118219.407192,492284.568472,...,2404.086963,1282.133980,8853.172865,175149.860322,15803.038758,139520.049866,1005.332985,25682.124607,14427.265779,1098.628983
1230,1.549313e+07,3244.436950,2484.005962,24494.315625,2599.253960,18103.882723,OAS2_0001_MR2_mpr-1,2249.736966,115559.442233,498485.322377,...,2426.724963,1212.161981,8839.452865,167995.566431,15588.663762,139861.677861,980.636985,24790.324621,13792.715789,1063.299984
219,1.546960e+07,3190.242951,2425.352963,24648.665623,2594.108960,14766.149774,OAS2_0001_MR1_mpr-3,2187.996967,121772.201138,495058.752429,...,2461.367962,1290.708980,9081.267861,180982.918232,15705.969760,140768.912847,997.443985,26277.572598,14516.445778,1194.325982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,1.577531e+07,1185.064982,1619.988975,19963.285695,1013.564985,1225.195981,OAS2_0186_MR2_mpr-4,1640.568975,90258.733620,399392.623892,...,1784.628973,1140.474983,7479.457886,143319.460808,12641.607807,149372.381716,805.363988,18012.987725,8594.207869,852.354987
714,1.576497e+07,1157.967982,1566.823976,20443.485687,959.713985,1232.398981,OAS2_0186_MR1_mpr-2,1723.574974,91764.160597,401563.813859,...,1771.937973,1164.141982,7519.245885,145811.698770,12658.071806,150548.871698,770.034988,18068.210724,8668.981867,777.237988
99,1.576301e+07,1146.648982,1539.383976,20565.250686,932.959986,1215.934981,OAS2_0186_MR1_mpr-1,1714.656974,91991.912593,401564.842859,...,1767.478973,1161.740982,7444.814886,146278.864763,12662.187806,150676.810696,747.396989,18231.478721,8786.630866,783.411988
348,1.577132e+07,1253.664981,1581.915976,20511.742686,979.607985,1551.388976,OAS2_0186_MR3_mpr-1,1753.415973,91178.316606,394133.747973,...,1777.082973,1118.865983,7661.247883,144650.300788,12494.460809,151302.099686,809.822988,17993.093725,8848.027865,853.383987


In [61]:
new_final_df_all = new_final_df_all .drop(columns = ['Unknown 0 0 0 0','choroid-plexus 0 200 200 0','choroid-plexus 0 200 221 0','WM-hypointensities 200 70 255 0']) 

In [63]:
#new_final_df_all.to_csv("oasis2_alldata_afterswift_fastsurfer_20240427.csv")